## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
# vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
# vacation_df.head()
# 1. Import the WeatherPy_database.csv file. 
vacation_df = pd.read_csv("../Vacation_Search/Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,2,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
1,6,Sao Joao Da Barra,BR,77.09,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas
2,10,Kirakira,SB,78.46,overcast clouds,-10.4544,161.9205,SanBiz Lodge
3,14,Sambava,MG,78.37,light rain,-14.2667,50.1667,Melrose
4,16,Saint-Pierre,RE,78.48,broken clouds,-21.3393,55.4781,Lindsey Hôtel


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [10]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Set the city as the index to easily retrieve the city 
vacation_city_df = vacation_df.set_index("City")
vacation_city_df.head()

,City_ID,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
City,,,,,,,
Puerto Ayora,2,EC,82.36,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
Sao Joao Da Barra,6,BR,77.09,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas
Kirakira,10,SB,78.46,overcast clouds,-10.4544,161.9205,SanBiz Lodge
Sambava,14,MG,78.37,light rain,-14.2667,50.1667,Melrose
Saint-Pierre,16,RE,78.48,broken clouds,-21.3393,55.4781,Lindsey Hôtel


In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
vacation_start = vacation_city_df.loc["Port Hedland"]
vacation_end = vacation_city_df.loc["Port Hedland"]
vacation_stop1 = vacation_city_df.loc["Karratha"]
vacation_stop2 = vacation_city_df.loc["Carnarvon"]
vacation_stop3 = vacation_city_df.loc["Kwinana"]

In [8]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start_1 = vacation_start['Lat':'Lng'].to_numpy()
end_1 = vacation_end['Lat':'Lng'].to_numpy()
stop1_1 = vacation_stop1['Lat':'Lng'].to_numpy()
stop2_1 = vacation_stop2['Lat':'Lng'].to_numpy()
stop3_1 = vacation_stop3['Lat':'Lng'].to_numpy()

start = tuple(start_1)
end = tuple(end_1)
stop1 = tuple(stop1_1)
stop2 = tuple(stop2_1)
stop3 = tuple(stop3_1)

In [9]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
portviaport = gmaps.directions_layer(
        start, end, waypoints=[stop1,stop2,stop3],
        travel_mode='DRIVING')
fig.add_layer(portviaport)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_end,vacation_stop1,vacation_stop2,vacation_stop3],axis=1, ignore_index=True)
# Switch the rows and columns 
# https://note.nkmk.me/en/python-pandas-t-transpose/
itinerary_df_t = itinerary_df.transpose()
# Add back in the city column 
itinerary_df_t.insert(1,"City", ["Port Hedland", "Port Hedland","Karratha", "Carnarvon", "Kwinana",], True)
itinerary_df_t.head()

,City_ID,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,126,Port Hedland,AU,84.25,few clouds,-20.3167,118.5667,The Esplanade Hotel (Accommodation section clo...
1,126,Port Hedland,AU,84.25,few clouds,-20.3167,118.5667,The Esplanade Hotel (Accommodation section clo...
2,488,Karratha,AU,84.45,clear sky,-20.7377,116.8463,ibis Styles Karratha
3,81,Carnarvon,AU,80.67,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
4,226,Kwinana,AU,72.93,clear sky,-32.25,115.7667,Rockingham Apartments


In [13]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df_t.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
local = itinerary_df_t[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map.
# 4a. Add a marker layer for each city to the map.
locations = local[["Lat", "Lng"]]
max_temp = itinerary_df_t["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))